In [1]:
import functools

In [2]:
class Tile:
    id = 0
    grid = None
    values = None
    neighbors = None
    def __init__(self, id, grid, neighbors = None):
        self.id = id
        self.grid = grid
        if neighbors is None:
            self.neighbors = set()
        else:
            self.neighbors = set(neighbors)
        self.detect_values()
    def update_grid(self, grid):
        self.grid = grid
        self.detect_values()
    def detect_values(self):
        top = ''.join(self.grid[0])
        bottom = ''.join(self.grid[-1])
        left = ''.join(line[0] for line in self.grid)
        right = ''.join(line[-1] for line in self.grid)
        self.values = [top, bottom, left, right]
    def __repr__(self):
        parts = [''.join(line) for line in self.grid]
        return "\n".join(parts)
    def __str__(self):
        return repr(self)
    def __hash__(self):
        return hash(self.id)

In [3]:
def get_input(fname="input.txt"):
    tiles = []
    with open(fname) as f:
        lines = f.readlines()
        loops = 1 + (len(lines) - 1) // 12
        for i in range(loops):
            id = int(lines[i * 12].strip().strip(":").split(" ")[-1])
            grid = [list(l.strip()) for l in lines[i * 12 + 1: i * 12 + 11]]
            tiles.append(Tile(id, grid))
    return tiles

In [4]:
def flip_value(v):
    return ''.join(reversed(v))

In [5]:
def arrage_image(tiles):
    changed = True
    while changed:
        changed = False
        edges_to_tiles = {}
        for tile in tiles.values():
            if len(tile.neighbors) == 4:
                continue
            for edge in tile.values:
                edge = min(edge, flip_value(edge))
                if edge not in edges_to_tiles:
                    edges_to_tiles[edge] = set()
                edges_to_tiles[edge].add(tile)
        for edge, tset in edges_to_tiles.items():
            if len(tset) == 2:
                changed = True
                for t in tset:
                    t.neighbors.add((tset - { t }).pop())
                    if edge in t.values:
                        t.values.remove(edge)
                    else:
                        t.values.remove(flip_value(edge))
    corners = set()
    for t in tiles.values():
        if len(t.neighbors) == 2:
            corners.add(t)
    return [t.id for t in corners]

In [6]:
test_data = { t.id: t for t in get_input("test.txt") }

In [7]:
arrage_image(test_data)

[3079, 1171, 2971, 1951]

In [8]:
input_data = { t.id: t for t in get_input("input.txt") }

In [9]:
result = arrage_image(input_data)

In [10]:
functools.reduce(lambda a, b: a * b, result)

45443966642567

In [11]:
def flip_horizontally(grid):
    return [list(reversed(line)) for line in grid]

def flip_vertically(grid):
    return [list(line) for line in reversed(grid)]

# https://stackoverflow.com/questions/53895650/matrix-90-degree-clockwise-rotation-using-list-in-python/60331181
def rotate_clockwise(grid):
    return [list(reversed(col)) for col in zip(*grid)]

# https://stackoverflow.com/questions/53250821/in-python-how-do-i-rotate-a-matrix-90-degrees-counterclockwise
def rotate_counterclockwise(grid):
    return [[grid[j][i] for j in range(len(grid))] for i in range(len(grid[0])-1,-1,-1)]

def copy(grid):
    return [list(line) for line in grid]

In [12]:
TOP, BOTTOM, LEFT, RIGHT = (0, 1, 2, 3)

In [13]:
def transform_tile_to_match_right(fixed_tile, other_tile):
    if other_tile is None:
        return
    value = fixed_tile.values[RIGHT]
    if value == other_tile.values[LEFT]:
        # already matching
        transform = copy
    elif value == flip_value(other_tile.values[LEFT]):
        # right side matches other's flipped left side -> flip vertically
        transform = flip_vertically
    elif value == flip_value(other_tile.values[TOP]):
        # right side matches other's flipped top side -> rotate counter clockwise
        transform = rotate_counterclockwise
    elif value == other_tile.values[TOP]:
        # right side matches other's top side -> rotate counter clockwise and flip vertically
        transform = lambda grid: flip_vertically(rotate_counterclockwise(grid))
    elif value == other_tile.values[RIGHT]:
        # right side matches other's right side -> flip horizontally
        transform = flip_horizontally
    elif value == flip_value(other_tile.values[RIGHT]):
        # right side matches other's reversed right side -> flip horizontally and vertically
        transform = lambda grid: flip_vertically(flip_horizontally(grid))
    elif value == other_tile.values[BOTTOM]:
        # right side matches other's bottom side -> rotate clockwise
        transform = rotate_clockwise
    elif value == flip_value(other_tile.values[BOTTOM]):
        # right side matches other's flipped bottom side -> rotate clockwise and flip vertically
        transform = lambda grid: flip_vertically(rotate_clockwise(grid))
    else:
        print("Shouldn't happen")
        return
    other_tile.update_grid(transform(other_tile.grid))
    # also transform downward tile
    transform_tile_to_match_down(other_tile, find_match(other_tile, BOTTOM))

def transform_tile_to_match_down(fixed_tile, other_tile):
    if other_tile is None:
        return
    value = fixed_tile.values[BOTTOM]
    if value == other_tile.values[TOP]:
        # already matching
        transform = copy
    elif value == flip_value(other_tile.values[TOP]):
        # bottom side matches other's flipped top side -> flip horizontally
        transform = flip_horizontally
    elif value == other_tile.values[BOTTOM]:
        # bottom side matches other's bottom side -> flip vertically
        transform = flip_vertically
    elif value == flip_value(other_tile.values[BOTTOM]):
        # bottom side matches other's flipped bottom side -> flip horizontally and vertically
        transform = lambda grid: flip_vertically(flip_horizontally(grid))
    elif value == other_tile.values[LEFT]:
        # bottom side matches other's left side -> rotate clockwise and flip horizontally
        transform = lambda grid: flip_horizontally(rotate_clockwise(grid))
    elif value == flip_value(other_tile.values[LEFT]):
        # bottom side matches other's flipped left side -> rotate clockwise
        transform = rotate_clockwise
    elif value == other_tile.values[RIGHT]:
        # bottom side matches other's right side -> rotate counter clockwise
        transform = rotate_counterclockwise
    elif value == flip_value(other_tile.values[RIGHT]):
        # bottom side matches other's flipped right side -> rotate counter clockwise and flip horizontally
        transform = lambda grid: flip_horizontally(rotate_counterclockwise(grid))
    else:
        print("Shouldn't happen")
        return
    other_tile.update_grid(transform(other_tile.grid))

In [14]:
def find_match(tile, direction=RIGHT):
    if tile is None:
        return None
    value = tile.values[direction]
    for neighbor in tile.neighbors:
        for v in neighbor.values:
            if value == v or value == flip_value(v):
                return neighbor
    return None

In [15]:
def rotate_corner(corner):
    directions = set()
    for d in { TOP, BOTTOM, LEFT, RIGHT }:
        if find_match(corner, d):
            directions.add(d)
    if directions == { BOTTOM, RIGHT }:
        return
    elif directions == { BOTTOM, LEFT }:
        corner.update_grid(flip_horizontally(corner.grid))
    elif directions == { TOP, RIGHT }:
        corner.update_grid(flip_vertically(corner.grid))
    elif directions == { TOP, LEFT }:
        corner.update_grid(flip_horizontally(flip_vertically(corner.grid)))

In [16]:
def get_image(tiles, corner):
    for t in tiles.values():
        t.detect_values()
    grid = [[]]
    # go right & down
    current = tiles[corner]
    # rotate corner to be able to traverse
    rotate_corner(current)
    dwn = find_match(current, BOTTOM)
    dwn.neighbors.remove(current)
    transform_tile_to_match_down(current, dwn)
    matched = 0
    direction = RIGHT
    while matched != len(tiles):
        grid[-1].append(current)
        matched += 1
        if len(current.neighbors) == 0:
            break
        rgt = find_match(current, RIGHT)
        if rgt is None:
            current = dwn
            grid.append([])
            dwn = find_match(current, BOTTOM)
            if dwn is not None:
                transform_tile_to_match_down(current, dwn)
                dwn.neighbors.remove(current)
        else:
            transform_tile_to_match_right(current, rgt)
            rgt.neighbors.remove(current)
            current.neighbors.remove(rgt)
            current = rgt
    return grid[:-1]

In [17]:
def assemble_image(image_grid):
    image = []
    for l in image_grid:
        for i in range(1, len(l[0].grid) - 1):
            row = []
            for tile in l:
                row += tile.grid[i][1:-1]
            image.append(''.join(row))
    return '\n'.join(image)

In [18]:
test_data = { t.id: t for t in get_input("test.txt") }
arrage_image(test_data)
test_image_grid = get_image(test_data, corner=1951)

In [19]:
print(assemble_image(test_image_grid))

.#.#..#.##...#.##..#####
###....#.#....#..#......
##.##.###.#.#..######...
###.#####...#.#####.#..#
##.#....#.##.####...#.##
...########.#....#####.#
....#..#...##..#.#.###..
.####...#..#.....#......
#..#.##..#..###.#.##....
#.####..#.####.#.#.###..
###.#.#...#.######.#..##
#.####....##..########.#
##..##.#...#...#.#.#.#..
...#..#..#.#.##..###.###
.#.#....#.##.#...###.##.
###.#...#..#.##.######..
.#.#.###.##.##.#..#.##..
.####.###.#...###.#..#.#
..#.#..#..#.#.#.####.###
#..####...#.#.#.###.###.
#####..#####...###....##
#.##..#..#...#..####...#
.#.###..##..##..####.##.
...###...##...#...#..###


In [20]:
monster = """
                  # 
#    ##    ##    ###
 #  #  #  #  #  #   
""".split("\n")[1:-1]

In [21]:
flipped_monster = flip_horizontally(monster)

In [22]:
monster_patterns = [
    monster,
    rotate_clockwise(monster),
    rotate_clockwise(rotate_clockwise(monster)),
    rotate_counterclockwise(monster),
    flipped_monster,
    rotate_clockwise(flipped_monster),
    rotate_clockwise(rotate_clockwise(flipped_monster)),
    rotate_counterclockwise(flipped_monster),
]

In [23]:
input_data = { t.id: t for t in get_input("input.txt") }
corners = arrage_image(input_data)
image_grid = get_image(input_data, corner=corners[0])

In [24]:
test_image = assemble_image(test_image_grid)

In [25]:
print(test_image)

.#.#..#.##...#.##..#####
###....#.#....#..#......
##.##.###.#.#..######...
###.#####...#.#####.#..#
##.#....#.##.####...#.##
...########.#....#####.#
....#..#...##..#.#.###..
.####...#..#.....#......
#..#.##..#..###.#.##....
#.####..#.####.#.#.###..
###.#.#...#.######.#..##
#.####....##..########.#
##..##.#...#...#.#.#.#..
...#..#..#.#.##..###.###
.#.#....#.##.#...###.##.
###.#...#..#.##.######..
.#.#.###.##.##.#..#.##..
.####.###.#...###.#..#.#
..#.#..#..#.#.#.####.###
#..####...#.#.#.###.###.
#####..#####...###....##
#.##..#..#...#..####...#
.#.###..##..##..####.##.
...###...##...#...#..###


In [26]:
def detect_monsters(image, monster):
    image = [list(l) for l in image.split("\n")]
    monster_width = len(monster[0])
    monster_height = len(monster)
    num_monsters = 0
    for i in range(len(image) - monster_height):
        for j in range(len(image[0]) - monster_width):
            is_monster = True
            for k in range(monster_height):
                if not is_monster:
                    break
                for l in range(monster_width):
                    if monster[k][l] == '#' and image[i + k][j + l] == '.':
                        is_monster = False
                        break
            if is_monster:
                num_monsters += 1
                for k in range(monster_height):
                    for l in range(monster_width):
                        if monster[k][l] == '#':
                            image[i + k][j + l] = 'o'
    return '\n'.join(''.join(l) for l in image), num_monsters

In [27]:
for pattern in monster_patterns:
    detected_image, detected_monsters = detect_monsters(test_image, pattern)
    if detected_monsters > 0:
        print(detected_image)
        print(sum(c == '#' for c in detected_image))

.#.#..#.##...#.##..#####
###....#.#....#..o......
##.o#.###.#.#..###o##...
###.o####...#.#####.#..#
##.#....#.##.####...#.##
...########.#....#o###.#
....o..#...##..#.o.###..
.##o#...#..#.....o......
#..o.##..#..###.#.o#....
#.##o#..#.####.#.#.###..
###.#.#...#.######.#..##
#.####....##..####o###.#
##..o#.#...#...#.o.#.#..
...o..#..#.#.##..o##.###
.#.o....#.##.#...#o#.##.
###.o...#..#.##.######..
.#.#.###.##.##.#..#.##..
.####.###.#...###.o..#.#
..#.o..#..#.#.#.#o##.###
#..o###...#.#.#.oo#.###.
##oo#..#####...##o....##
#.#o..#..#...#..####...#
.#.###..##..##..####.##.
...###...##...#...#..###
273


In [28]:
image = assemble_image(image_grid)

In [29]:
for pattern in monster_patterns:
    detected_image, detected_monsters = detect_monsters(image, pattern)
    if detected_monsters > 0:
        print(sum(c == '#' for c in detected_image))
        print(detected_image)

1607
..#.##........#....#####...............#.#.#...................#..#.........#.#....#.##...#.....
....#.#....##......#....#.........#..........#.#.....#.....#.........#...#..#....###.#.#.#..#...
...........#.##.#.......#..#..........##..#.......#.....#....#....#...#....##.#.#.#.#...#.......
...#......#.......#...#.....o.........##..#.......#..............#..####..........#....#...#....
......#....##.#......##.....oo.##.####....#....#..#...##.#..#..##...#......##....#..##.......#..
#....#...#....#.............o...................#..#.......#..#......##...#.......#......##.....
...#.....##.....#....##.#.#o...#..............................#....#.....##.....###.#.#.##.....#
..#.....#.....##.#................#................#...............##....#..#...#........##.....
.........#..#........#........#.......##.###...................#..#...#......#........##........
..#...#.o.##..#...#........o...#.#.#.#.....o.#.#...###.......#..........#.......................
#.#....#oo#.#..#.....#..#